# Function Calling

[Doc OpenAI Function Calling](https://platform.openai.com/docs/guides/function-calling)


[Beginner Tutorial](https://www.youtube.com/watch?v=aqdWSYWC_LI)

Allows yo connect models from OpenAI to external tools and systems. 

In this example we will see how function calling works from a simple perspective, just to see how it works and test the functionallity.

In [3]:
# import libraries

import os
import json
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # Load environment variables from .env file
openai.api_key = os.environ["OPENAI_API_KEY"] # Set OpenAI API key

In [4]:
# STEP 1
# In this step, we need to choose a function in our codebase that we want to enable the model to generate arguments for.
# Here, we simulate an API to get weather data, and we want the model to be able to call this function.

def get_current_weather(location, unit="celsius"):
    "Get the current weather for a given location"
    weather_info = {
        "location": location, 
        "temperture": 25,
        "unit": unit,
        "forecast": "sunny"
    
    }
    return json.dumps(weather_info)

In [5]:
# STEP 2
# Now that we have a function that we want to enable the model to call, we need to define "function definition".
# This step describes: what the function does, when it should be called and what parameters are required to call the funcion.
# Should be in  a JSON schema.

functions = [
    {
        "name": "get_current_weather", # Name of the API function
        "description": "Get the current weather in a given location", # Description of the function
        # The parameters required to call the function
        "parameters": {
            "type": "object", # The type of the parameters
            "properties": { # The properties of the parameters
                "location": { # The location parameter
                    "type": "string", # The type of the parameter
                    "description": "The city and state to find the current weather.", # The description of the parameter
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}, # The unit parameter
            }, 
            "required": ["location"], # The required parameters
        },
    }
]

In [6]:
# STEP 3
# Let´s define the message to send to the model.

messages = [
    {
        "role": "user", 
        "content": "What is the current weather in Merida, Spain?"
    }
]

In [7]:
# STEP 4
# Now we can call the OpenAI API to generate the arguments for the function.

response = openai.ChatCompletion.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions
)

In [8]:
print(response)

{
  "id": "chatcmpl-AIdM5yShJ6OEmqWvfxdM15199ldSO",
  "object": "chat.completion",
  "created": 1729004549,
  "model": "gpt-4o-mini-2024-07-18",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\"location\":\"Merida, Spain\"}"
        },
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 80,
    "completion_tokens": 18,
    "total_tokens": 98,
    "prompt_tokens_details": {
      "cached_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0
    }
  },
  "system_fingerprint": "fp_e2bde53e6e"
}


In [9]:
response_message = response["choices"][0]["message"] # Get the response message

In [10]:
response_message

<OpenAIObject at 0x1df5b2be870> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\"location\":\"Merida, Spain\"}"
  },
  "refusal": null
}

In [11]:
response_message["content"]

In [12]:
response_message["function_call"] # we can see the model using the function we defined

<OpenAIObject at 0x1df5b2be8d0> JSON: {
  "name": "get_current_weather",
  "arguments": "{\"location\":\"Merida, Spain\"}"
}

In [13]:
json.loads(response_message["function_call"]["arguments"])

{'location': 'Merida, Spain'}

In [14]:
args = json.loads(response_message["function_call"]["arguments"])

In [15]:
get_current_weather(args) # Call the function with the arguments generated by the model

'{"location": {"location": "Merida, Spain"}, "temperture": 25, "unit": "celsius", "forecast": "sunny"}'

In [16]:
# Let´s play with the model and some function calling arguments
# Start using the function call argument to auto. Apparently this is not recommended anymore, but it works.
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
    function_call="auto", 
)
print(response)

{
  "id": "chatcmpl-AIdMqhXe4HbVnOTkMSFUdyDrv7b8i",
  "object": "chat.completion",
  "created": 1729004596,
  "model": "gpt-4o-mini-2024-07-18",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 71,
    "completion_tokens": 10,
    "total_tokens": 81,
    "prompt_tokens_details": {
      "cached_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0
    }
  },
  "system_fingerprint": "fp_e2bde53e6e"
}


In [17]:
# Start using the function call argument
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
    function_call="none", 
)
print(response)

{
  "id": "chatcmpl-AIdMv8rHktpONlXD3WeCwVgfVPEXe",
  "object": "chat.completion",
  "created": 1729004601,
  "model": "gpt-4o-mini-2024-07-18",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 72,
    "completion_tokens": 9,
    "total_tokens": 81,
    "prompt_tokens_details": {
      "cached_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0
    }
  },
  "system_fingerprint": "fp_e2bde53e6e"
}


In [18]:
# Start using the function call argument
messages = [
    {
        "role": "user",
        "content": "What is the current weather in Merida, Spain?",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
    function_call="none"
)
print(response)

{
  "id": "chatcmpl-AIdN7FmqFGrrybIrIPkLji4HZq2LV",
  "object": "chat.completion",
  "created": 1729004613,
  "model": "gpt-4o-mini-2024-07-18",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "I will check the current weather in Merida, Spain for you. One moment please.",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 81,
    "completion_tokens": 18,
    "total_tokens": 99,
    "prompt_tokens_details": {
      "cached_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0
    }
  },
  "system_fingerprint": "fp_e2bde53e6e"
}


In [19]:
# Start using the function call argument
messages = [
    {
        "role": "user",
        "content": "What is the current weather in Merida, Spain?",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}
)
print(response)

{
  "id": "chatcmpl-AIdNBk3jp3ChStQyC2JpCt8hvKWU0",
  "object": "chat.completion",
  "created": 1729004617,
  "model": "gpt-4o-mini-2024-07-18",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\"location\":\"Merida, Spain\"}"
        },
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 90,
    "completion_tokens": 8,
    "total_tokens": 98,
    "prompt_tokens_details": {
      "cached_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0
    }
  },
  "system_fingerprint": "fp_e2bde53e6e"
}
